In [18]:
import pandas as pd
df = pd.read_csv('SBAnational.csv') 

/tmp/ipykernel_7398/2887181283.py:2: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('SBAnational.csv')


In [24]:
def daterange_between(a_start, a_end, b_start, b_end):
    return np.minimum(a_end, b_end) > np.maximum(a_start, b_start)

In [26]:
import numpy as np
cols = ['DisbursementGross', 'BalanceGross', 'ChgOffPrinGr', 'GrAppv', 'SBA_Appv']
df[cols] = df[cols].replace({'\$': '', ' ': '', ',':''}, regex=True).astype(float)
df['DisbursementDate'] = pd.to_datetime(df['DisbursementDate'][0])
df = df.assign(
    RealEstate = lambda x: pd.cut(x['Term'], bins=[0, 240, 999999], labels=[0,1]),
    Portion = lambda x: x['SBA_Appv'] / x['DisbursementGross'],
    EndDate = lambda x: x['DisbursementDate'] + x['Term'].astype('timedelta64[M]'),
    Recession = lambda x: daterange_between(np.datetime64('2007-12-01'),np.datetime64('2009-01-01'), x['DisbursementDate'], x['EndDate']).astype(int)
)
df.iloc[0]


LoanNr_ChkDgt                 1000014003
Name                      ABC HOBBYCRAFT
City                          EVANSVILLE
State                                 IN
Zip                                47711
Bank                    FIFTH THIRD BANK
BankState                             OH
NAICS                             451120
ApprovalDate                   28-Feb-97
ApprovalFY                          1997
Term                                  84
NoEmp                                  4
NewExist                             2.0
CreateJob                              0
RetainedJob                            0
FranchiseCode                          1
UrbanRural                             0
RevLineCr                              N
LowDoc                                 Y
ChgOffDate                           NaN
DisbursementDate     1999-02-28 00:00:00
DisbursementGross                60000.0
BalanceGross                         0.0
MIS_Status                         P I F
ChgOffPrinGr    

ANN 
Artificial neural networks

In [51]:
from keras.utils.np_utils  import to_categorical
from sklearn.model_selection import train_test_split

x = df[['RealEstate', 'Portion', 'Recession']].to_numpy()
y = df['MIS_Status'].apply(lambda x: int((x=='CHGOFF')))

y = to_categorical(y, num_classes=None) 

# Testing wit train test67% 33%
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.33,random_state=42)



array([[0.        , 1.        , 1.        ],
       [1.        , 0.85      , 1.        ],
       [0.        , 0.28930497, 0.        ],
       ...,
       [0.        , 0.9       , 0.        ],
       [0.        , 0.5       , 0.        ],
       [0.        , 0.25045583, 0.        ]])

Creating a ANN model